In [1]:
import requests
import json
from tqdm import tqdm

import pandas as pd

In [45]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [17]:
#https://geoscience.data.qld.gov.au/api/action/package_search?q=geochem&rows=1000&start=0
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [39]:
# set the API endpoint
api = 'https://geoscience.data.qld.gov.au/api/action/'
# construct our query
query = api + 'package_search?q=geochemistry&rows=1000'
# make the get request and store it in the response object
response = requests.get(query, headers=headers)
# view the payload as JSON
#print(response.text)

if 1 == 1:
    #json_response = response.json()
    json_response = json.loads(response.text)

    #and get a count of results we can retrieve
    total_results = json_response['result']['count']
    print(total_results)
    
    #print(json_response['result']['results'][0])

    #hack without understanding/researching the api too much
    getloop = int(total_results/1000) + 1

    if getloop > 1:
        results_list = []
        for i in range(getloop):
            loopq = query + "&start=" + str(i*1000)
            print(loopq)
            response = requests.get(loopq, headers=headers)
            #json_response = response.json()
            json_response = json.loads(response.text)

            results_list.append(json_response)


12125
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=0
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=1000
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=2000
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=3000
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=4000
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=5000
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=6000
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=7000
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=8000
https://geoscience.data.qld.gov.au/api/action/package_search?q=geochemistry&rows=1000&start=9000
https://geoscience.data.qld

In [40]:
#look at a result for the structure
results_list[0]['result']['results'][887]

{'owner_org': '1461ac2d-f813-4c97-9e3b-3900a23b0ecb',
 'maintainer': None,
 'creator': 'geological-survey-of-queensland',
 'relationships_as_object': [],
 'private': False,
 'maintainer_email': None,
 'num_tags': 1,
 'GeoJSONextent': '{"type": "Polygon", "coordinates": [[[147.75108710012444, -21.331793143854668], [147.83441875601932, -21.331793410854686], [147.8344195440194, -21.41512639974968], [147.75108796712448, -21.415126266749667], [147.75108710012444, -21.331793143854668]]]}',
 'id': '94a4559c-1191-4e91-898f-756237f27151',
 'metadata_created': '2019-06-05T10:05:20.964289',
 'metadata_modified': '2020-08-25T10:10:18.367286',
 'author': None,
 'author_email': None,
 'extra:access_rights': 'http://linked.data.gov.au/def/data-access/open',
 'state': 'active',
 'version': None,
 'extra:contact_uri': 'GSQOpenData@dnrme.qld.gov.au',
 'spatial': '{"type": "Polygon", "coordinates": [[[147.75108710012444, -21.331793143854668], [147.83441875601932, -21.331793410854686], [147.8344195440194,

In [98]:
#each one has organisation - don't really need
#id, , author, spatial, type is useful
#resources - a list of dicts, 
#eg resource:description:
#resource description dictionary
#a resource id, resource:size, resource:, download_url, name, url, resource_type etc

In [99]:
#will try and make a dataframe
#but using the batched results, getting the ids
#turning the resource dictionaries into dataframes, concatenating
#then concatenating back at the end

In [41]:
result_ids = []
for batch in tqdm(results_list, total=len(results_list) ):
    results = batch['result']['results']
    for r in results:
        rid = r['id']
        rauthor = r['author']
        rspatial = r['spatial']
        rtype = r['type']
        
        if len(r['resources']) > 0:
            part_list = []
            for part in r['resources']:
                #modify the columns here to include what you want
                df = pd.DataFrame([part])
                df['rid'] = rid
                df['rauthor'] = rauthor
                df['rspatial'] = rspatial
                df['type'] = rtype
                part_list.append(df)
                allparts = pd.concat(part_list)
                
            result_ids.append(allparts)

dfDatasets = pd.concat(result_ids)            

100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [40:32<00:00, 187.08s/it]


In [42]:
dfDatasets.to_csv(r'H:\GSQ_Geochemistry_Datasets.csv', index=False)

In [63]:
#this is what a default query would look like
query_default = api + 'package_search?q=seismic

response = requests.get(query_default)
json_response = response.json()

print(json_response['result']['count'])
print(len(json_response['result']['results']) )
for x in json_response['result']['results']:
    print(x["id"])

10
4267
aaa7c8c3-d69b-4ae6-bc4b-2542301b7e6e
39bc2c60-d727-4383-b82d-6af7fdf8cf7b
2f3d761c-6506-438a-934a-87e48cc8d25e
8a1f58c4-2e61-44a5-a573-794396652ea6
55a1c3a1-107f-4a65-9bec-bef9e6eb8d6b
8157408f-83af-4515-9279-f502ae16c155
37a2fb0c-49c5-444a-a7da-4947ae9dad94
3897db72-0d38-4f11-aad4-8fa8907d20bb
4d5ae331-3cfb-4042-a4b5-1955c135cace
30b6b5fc-ba25-4bfe-a76b-c35d72ba07e0


Check how many are of type seismic, not report

In [102]:
dfDatasets.loc[dfDatasets['type'] == 'seismic']['rid'].unique().size

1290

In [103]:
dfDatasets.loc[dfDatasets['type'] == 'seismic']

,resource:description,cache_last_updated,package_id,datastore_contains_all_records_of_source_file,datastore_active,id,size,resource:format,resource:size,download_url,...,rid,rauthor,rspatial,type,original_url,resource_id,set_url_type,ignore_hash,ckan_url,task_created
0,95332_GEOMETRY_SHP,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-spa303286-ss095332,None,ZIP,,https://geoscience.data.qld.gov.au/dataset/39b...,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,Data migrated from source,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-doc530683-ss095332,None,JSON,1 B,https://geoscience.data.qld.gov.au/dataset/39b...,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,KOGAN SEISMIC SURVEY 2012 Seismic Survey: 9533...,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-doc1040204-ss095332,None,ZIP,935.93 MB,NaN,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,KOGAN SEISMIC SURVEY 2012 Seismic Survey: 9533...,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-doc1040205-ss095332,None,ZIP,184.41 MB,NaN,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,KOGAN SEISMIC SURVEY 2012 Seismic Survey: 9533...,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-doc1040206-ss095332,None,SHP,37.59 KB,NaN,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Line_96-GYR_SEGD,None,046e9982-89ff-4297-ba5b-baca0bceefaf,False,False,geo-doc1302238-ss095054,None,ZIP,89.84 MB,https://geoscience.data.qld.gov.au/dataset/046...,...,046e9982-89ff-4297-ba5b-baca0bceefaf,None,"{""type"":""Polygon"",""coordinates"":[[[140.9778903...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,Line_96-GYZ_SEGD,None,046e9982-89ff-4297-ba5b-baca0bceefaf,False,False,geo-doc1302239-ss095054,None,ZIP,499.81 MB,https://geoscience.data.qld.gov.au/dataset/046...,...,046e9982-89ff-4297-ba5b-baca0bceefaf,None,"{""type"":""Polygon"",""coordinates"":[[[140.9778903...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,Line_96-GZH_SEGD,None,046e9982-89ff-4297-ba5b-baca0bceefaf,False,False,geo-doc1302240-ss095054,None,ZIP,441.86 MB,https://geoscience.data.qld.gov.au/dataset/046...,...,046e9982-89ff-4297-ba5b-baca0bceefaf,None,"{""type"":""Polygon"",""coordinates"":[[[140.9778903...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,Line_96-GZJ_SEGD,None,046e9982-89ff-4297-ba5b-baca0bceefaf,False,False,geo-doc1302241-ss095054,None,ZIP,443.2 MB,https://geoscience.data.qld.gov.au/dataset/046...,...,046e9982-89ff-4297-ba5b-baca0bceefaf,None,"{""type"":""Polygon"",""coordinates"":[[[140.9778903...",seismic,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
dfDatasets['url'].iloc[0]

'https://gsq-prod-ckan-horizon-public.s3.ap-southeast-2.amazonaws.com/Report%2f6342%2fDocument%2f443809%2f6342.zip'

In [108]:
dfDatasets['download_url'].loc[ (dfDatasets['type'] == 'seismic') & (dfDatasets['download_url'].str.contains('geoscience') )  ]

0    https://geoscience.data.qld.gov.au/dataset/39b...
0    https://geoscience.data.qld.gov.au/dataset/39b...
0    https://geoscience.data.qld.gov.au/dataset/8a1...
0    https://geoscience.data.qld.gov.au/dataset/8a1...
0    https://geoscience.data.qld.gov.au/dataset/13d...
                           ...                        
0    https://geoscience.data.qld.gov.au/dataset/046...
0    https://geoscience.data.qld.gov.au/dataset/046...
0    https://geoscience.data.qld.gov.au/dataset/046...
0    https://geoscience.data.qld.gov.au/dataset/046...
0    https://geoscience.data.qld.gov.au/dataset/046...
Name: download_url, Length: 2423, dtype: object

In [110]:
dfDatasets['download_url'].loc[ (dfDatasets['type'] == 'seismic') & (dfDatasets['download_url'].str.contains('geoscience') )  ].iloc[0]

'https://geoscience.data.qld.gov.au/dataset/39bc2c60-d727-4383-b82d-6af7fdf8cf7b/resource/geo-spa303286-ss095332/download/%252FSurvey%25252f95332%25252fGeometry%25252f303286%25252f95332_GEOMETRY_SHP.zip'

In [111]:
download_link = 'https://gsq-horizon.s3-ap-southeast-2.amazonaws.com/SEISMIC/2D_Surveys/2010-2019/95332_KOGAN_SEISMIC_SURVEY_2012/95332_PROCESSED.ZIP'
download_response = requests.get(download_link)



In [113]:
from zipfile import ZipFile

def extract_zip(input_zip):
    input_zip=ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}

In [130]:
len(download_response.content)

981394400

In [ ]:
import io, zipfile
download_link = 'https://gsq-horizon.s3-ap-southeast-2.amazonaws.com/SEISMIC/2D_Surveys/2010-2019/95332_KOGAN_SEISMIC_SURVEY_2012/95332_PROCESSED.ZIP'
download_response = requests.get(download_link)

zips = zipfile.ZipFile(io.BytesIO(response.content))


In [142]:
dfDatasets.shape

In [43]:
tw = pd.read_csv('https://geoscience.data.qld.gov.au/dataset/7c9c524c-7c40-456c-94bc-89b4f83ea240/resource/4e680199-0e38-45e2-b882-e3ec9b4e71c2/download/h_loc_town4.csv')

In [46]:
tw

,Unnamed: 0,Prospect_Code,Company,Sheet_Number,Hole_ID,Grid_ID,Country,State,Amg_N,Amg_E,RL_Regional,North,East,RL_Local,Latitude,Longitude,Location_Accuracy,Drilling_Type,Program_Type,Commodity_Type,Precollar_Depth,Final_Depth,Cover_Depth,Cover_Status,Cover_Age,BOCO_Depth,BOCO_Status,BOPO_Depth,BOPO_Status,Log_By,Perc_Start,Perc_End,Core_Start,Core_End,P_Drill_Co,D_Drill_Co,P_Drill_Rig,D_Drill_Rig,Completion_Status,Min_Occ,Geological_Unit_Code,Comment,Tenement_Type,Tenement,Report_Type,Report,Open_File,HasTextParams,Collar_ID,Locality,Water_Depth,Water_Table_Status,TagCharacter,Image_ID,Load_Number,Date_Updated,User_Updating,Update_Status,TS_Release,Spatial_Status,geometry,BIMNAME,BIMCODE,BLOCKDESC,BLOCKNO,SHAPE_Leng,SHAPE_Area
0,112952,STMAY,STRIKE,7765,MNPRC-01,AMG2,AUS,QLD,8222304.14,213320.93,NaN,NaN,NaN,NaN,-16.062367,144.320555,20.0,PERC,NaN,Au,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN,NaN,EPM,9444,CR,30632,Y,NaN,5004761,100km W of Peninsula Development Road,NaN,NaN,M,NaN,0,6/01/2016 0:00:00,PA,1,MM0003,NaN,POINT (144.3216544859121 -16.06085298504066),Townsville,TOWN,TOWN4,4,0.333331,0.006944
1,112953,STMAY,STRIKE,7765,MNPRC-02,AMG2,AUS,QLD,8222505.34,213020.80,NaN,NaN,NaN,NaN,-16.060515,144.317776,20.0,PERC,NaN,Au,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN,NaN,EPM,9444,CR,30632,Y,NaN,5004762,100km W of Peninsula Development Road,NaN,NaN,M,NaN,0,6/01/2016 0:00:00,PA,1,MM0003,NaN,POINT (144.3188755080479 -16.05900100594847),Townsville,TOWN,TOWN4,4,0.333331,0.006944
2,112954,STMAY,STRIKE,7765,MNPRC-03,AMG2,AUS,QLD,8222844.19,212990.57,NaN,NaN,NaN,NaN,-16.057451,144.317535,20.0,PERC,NaN,Au,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN,NaN,EPM,9444,CR,30632,Y,NaN,5004763,100km W of Peninsula Development Road,NaN,NaN,M,NaN,0,6/01/2016 0:00:00,PA,1,MM0003,NaN,POINT (144.3186344947746 -16.05593701922389),Townsville,TOWN,TOWN4,4,0.333331,0.006944


In [47]:
br = pd.read_csv('https://geoscience.data.qld.gov.au/dataset/c8510270-dbc6-48f9-95e6-a8936733b583/resource/02d2e353-c3c9-4633-a6c3-61827c793bf7/download/h_survey_bris2820.csv')

In [48]:
br

,Unnamed: 0,Prospect_Code,Company,Sheet_Number,Hole_ID,Depth_survey,Dip,Magnetic_Az,Regional_Azimuth,Local_Grid_Azimuth,Instrument,TagCharacter,Collar_ID,Survey_ID,HasTextParams,Load_Number,Date_Updated,User_Updating,Update_Status,BIMNAME,BIMCODE,BLOCKDESC,BLOCKNO,SHAPE_Leng,SHAPE_Area
0,128469,RGDA,RGC,9043,DA26,0.0,-90.0,0.0,NaN,NaN,C,M,5005873,5007679,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944
1,128470,RGDA,RGC,9043,DA27,0.0,-90.0,0.0,NaN,NaN,C,M,5005874,5007680,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944
2,128471,RGDA,RGC,9043,DA28,0.0,-90.0,0.0,NaN,NaN,C,M,5005875,5007681,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944
3,128472,RGDA,RGC,9043,DA29,0.0,-90.0,0.0,NaN,NaN,C,M,5005876,5007682,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944
4,128473,RGDA,RGC,9043,DA30,0.0,-90.0,0.0,NaN,NaN,C,M,5005877,5007683,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944
5,128474,RGDA,RGC,9043,DA31,0.0,-90.0,0.0,NaN,NaN,C,M,5005878,5007684,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944
6,128475,RGDA,RGC,9043,DA32,0.0,-90.0,0.0,NaN,NaN,C,M,5005879,5007685,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944
7,128476,RGDA,RGC,9043,DA33,0.0,-90.0,0.0,NaN,NaN,C,M,5005880,5007686,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944
8,128477,RGDA,RGC,9043,DA34,0.0,-90.0,0.0,NaN,NaN,C,M,5005881,5007687,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944
9,128478,RGDA,RGC,9043,DA35,0.0,-90.0,0.0,NaN,NaN,C,M,5005882,5007688,NaN,0,25/01/2016 0:00:00,PA,1,Brisbane,BRIS,BRIS2820,2820,0.33333,0.006944


In [49]:
cl = pd.read_csv('https://geoscience.data.qld.gov.au/dataset/b79c875b-4deb-439b-9e43-8d24eae94906/resource/866c2f3e-bfc4-48d2-9eb5-b95c5516ee57/download/h_sample_clon2040.csv')

In [50]:
cl

,Unnamed: 0,Prospect_Code,Company,Sheet_Number,Hole_ID,Sample,From_Depth,To_Depth,Data_Type,Drilling_Type,North,East,RL,Comment,Au,Au1,Au2,Au3,Au4,Cu,Pb,Zn,Ag,As,Bi,Mo,Mn,Fe,Ni,Co,Cr,V,Ba,Cd,Sn,Sb,Hg,Te,P,W,Zr,Ti,Mg,Th,U,Pt,Pd,S,F,Job_No,HasParam,HasTextParams,Duplicate_Flag,Preferred_Interval,Associated_Sample_Number,Pref_Au_Assay_Field,Wet_Dry_Flag,TagCharacter,Collar_ID,Sample_ID,Load_Number,Date_Updated,User_Updating,Update_Status,Old_Job,Lab_Code,Job_Date,Alias_Job,Dispatch_No,Location_Code,Description,Date_From,Date_To,BIMNAME,BIMCODE,BLOCKDESC,BLOCKNO,SHAPE_Leng,SHAPE_Area
0,1382848,TSPEBR,PNC,6853,1318,130016,78.0,80.0,INT,PERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,TSUNK81055,Y,N,NaN,NaN,NaN,NaN,NaN,T,77811,806994,11548,1/04/2010 9:07:52,dba,1,NaN,UNK,NaN,TSUNK0827,NaN,UNK,NaN,NaN,NaN,Cloncurry,CLON,CLON2040,2040,0.333326,0.006944
1,1382849,TSPEBR,PNC,6853,1318,130017,80.0,82.0,INT,PERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,4.0,NaN,NaN,NaN,NaN,TSUNK81060,Y,N,NaN,NaN,NaN,NaN,NaN,T,77811,806995,11548,1/04/2010 9:07:52,dba,1,NaN,UNK,NaN,TSUNK0832,NaN,UNK,NaN,NaN,NaN,Cloncurry,CLON,CLON2040,2040,0.333326,0.006944
2,1382850,TSPEBR,PNC,6853,1318,130018,82.0,84.0,INT,PERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,20.0,NaN,NaN,NaN,NaN,TSUNK81058,Y,N,NaN,NaN,NaN,NaN,NaN,T,77811,806996,11548,1/04/2010 9:07:52,dba,1,NaN,UNK,NaN,TSUNK0830,NaN,UNK,NaN,NaN,NaN,Cloncurry,CLON,CLON2040,2040,0.333326,0.006944
3,1382851,TSPEBR,PNC,6853,1318,130019,84.0,86.0,INT,PERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,14.0,NaN,NaN,NaN,NaN,TSUNK81062,Y,N,NaN,NaN,NaN,NaN,NaN,T,77811,806997,11548,1/04/2010 9:07:52,dba,1,NaN,UNK,NaN,TSUNK0834,NaN,UNK,NaN,NaN,NaN,Cloncurry,CLON,CLON2040,2040,0.333326,0.006944
4,1382852,TSPEBR,PNC,6853,1318,130020,86.0,88.0,INT,PERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,TSUNK81049,N,N,NaN,NaN,NaN,NaN,NaN,T,77811,806998,11548,1/04/2010 9:07:52,dba,1,NaN,UNK,NaN,TSUNK0821,NaN,UNK,NaN,NaN,NaN,Cloncurry,CLON,CLON2040,2040,0.333326,0.006944
5,1382853,TSPEBR,PNC,6853,1318,130021,88.0,90.0,INT,PERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,20.0,NaN,NaN,NaN,NaN,TSUNK81052,N,N,NaN,NaN,NaN,NaN,NaN,T,77811,806999,11548,1/04/2010 9:07:52,dba,1,NaN,UNK,NaN,TSUNK0824,NaN,UNK,NaN,NaN,NaN,Cloncurry,CLON,CLON2040,2040,0.333326,0.006944
6,1382854,TSPEBR,PNC,6853,1318,130022,90.0,92.0,INT,PERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,22.0,NaN,NaN,NaN,NaN,TSUNK81058,Y,N,NaN,NaN,NaN,NaN,NaN,T,77811,807000,11548,1/04/2010 9:07:52,dba,1,NaN,UNK,NaN,TSUNK0830,NaN,UNK,NaN,NaN,NaN,Cloncurry,CLON,CLON2040,2040,0.333326,0.006944
7,1382855,TSPEBR,PNC,6853,1318,130023,92.0,94.0,INT,PERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,TSUNK81058,Y,N,NaN,NaN,NaN,NaN,NaN,T,77811,807001,11548,1/04/2010 9:07:52,dba,1,NaN,UNK,NaN,TSUNK0830,NaN,UNK,NaN,NaN,NaN,Cloncurry,CLON,CLON2040,2040,0.333326,0.006944
8,1382856,TSPEBR,PNC,6853,1401,4180,60.0,62.0,INT,PERC,NaN,NaN,NaN,Geology given but not entered,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,4.0,NaN,NaN,NaN,NaN,TSUNK81088,Y,N,NaN,NaN,NaN,NaN,NaN,T,77812,807002,11582,1/04/2010 9:07:53,dba,1,NaN,UNK,NaN,TSUN

In [51]:
brwr = pd.read_csv('https://geoscience.data.qld.gov.au/dataset/88e09172-c2a6-4006-93e1-51d022662265/resource/c7d1bd02-1618-4969-811e-febc57c50f6c/download/s_wholerock_bris2038.csv')

In [52]:
brwr

,Unnamed: 0,Prospect_Code,Company,Sheet_Number,Sample,Data_Type,Hole_ID,From_Depth,To_Depth,Grid_ID,Locality,AMG_N,AMG_E,North,East,RL_Regional,RL_Local,Location_Accuracy,Latitude,Longitude,Country,State,Sampler,Sample_Date,Geological_Unit_Code,Regolith_Landform_Code,Lith1,Desc11,Desc12,Desc13,Desc14,AltQual11,AltQual12,AltQual13,AltType11,AltType12,AltType13,AltInt11,AltInt12,AltInt13,Min_Style11,Min_Style12,Min_Style13,Min_Style14,Min_Amount11,Min_Amount12,Min_Amount13,Min_Amount14,Min_Min11,Min_Min12,Min_Min13,Min_Min14,Lith2,Desc21,Desc22,Desc23,Lith_Desc,Age_No,Occurrence,Weathering_Code,Length,Width,Strike,Dip,Dip_Quadrant,Dip_Dir_Az,Vegetation,Ph_Ref,Ph_Run,Ph_Num,Min_Occurrence,Comment,SiO2,TiO2,Al2O3,FeOx_Total,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,H20_plus,H2O_minus,CO2,LOI,Rest,Total,Job_no,Sample_ID,HasAssayParam,HasLithParam,Duplicate,Associated_Sample_No,Tenement_Type,Tenement,Report_Type,Report,Open_File,TagCharacter,Image_ID,Load_Number,Date_Updated,User_Updating,Update_Status,TS_Release,Spatial_Status,Old_Job,Lab_Code,Job_Date,Alias_Job,Dispatch_No,Location_Code,Description,Date_From,Date_To,geometry,BIMNAME,BIMCODE,BLOCKDESC,BLOCKNO,SHAPE_Leng,SHAPE_Area
0,732,CPKY,CAP,9245,K-18,RC,NaN,NaN,NaN,AMG2,170km NW of Brisbane,7082570.0,380700.0,NaN,NaN,NaN,NaN,25,-26.371867,151.804193,AUS,QLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.77,2.27,25.6,NaN,18.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.11,NaN,NaN,ST39773,5000010,NaN,NaN,NaN,NaN,EPM,14639,CR,39053,Y,M,NaN,0,4/07/2016 0:00:00,PA,1,MM0007,NaN,NaN,ALS,23/04/2004,MMALS4206,NaN,BNE,NaN,NaN,NaN,POINT (151.8052631310128 -26.37028038554192),Brisbane,BRIS,BRIS2038,2038,0.33333,0.006944
